In [14]:
import dill

from CoRefHelper import parse_stanfordnlp_tagged_essays
from FindFiles import find_files
from Settings import Settings
from collections import defaultdict

CV_FOLDS = 5
DEV_SPLIT = 0.1

""" Begin Settings """
DATASET = "CoralBleaching"
PARTITION = "Training" # Training | Test
SCAN_LENGTH = 3
""" END Settings """

settings = Settings()
root_folder = settings.data_directory + DATASET + "/Thesis_Dataset/"
merged_predictions_folder = root_folder + "Predictions/CoRef/MergedTags/"

coref_root = root_folder + "CoReference/"
coref_folder = coref_root + PARTITION

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


## Load Merged Essays

In [15]:
##override this so we don't replace INFREQUENT words
#config["min_df"] = 0

if PARTITION.lower() == "training":
    merged_essays_fname =  "merged_essays_train.dill"
elif PARTITION.lower() == "test":
    merged_essays_fname = "merged_essays_test.dill"
else:
    raise Exception("Invalid partition: " + PARTITION)

merged_essays_fname = merged_predictions_folder + merged_essays_fname
with open(merged_essays_fname, "rb+") as f:
    tagged_essays = dill.load(f)

# map parsed essays to essay name
essay2tagged = {}
for e in tagged_essays:
    essay2tagged[e.name.split(".")[0]] = e

print("{0} training essays loaded from:\n{1}".format(len(tagged_essays), merged_essays_fname))

902 training essays loaded from:
/Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/Predictions/CoRef/MergedTags/merged_essays_train.dill


## Load CoRef Parsed Essays

In [16]:

# Load CoRef Parsed Essays
coref_files = find_files(coref_folder, ".*\.tagged")
print("{0} co-ref tagged files loaded".format(len(coref_files)))
essay2coref_tagged = parse_stanfordnlp_tagged_essays(coref_files)
assert len(coref_files) == len(tagged_essays)

902 co-ref tagged files loaded


## Validate Same Essays

In [17]:
# VALIDATE THE SAME SET OF ESSAYS
assert essay2tagged.keys() == essay2coref_tagged.keys()
intersect = set(essay2tagged.keys()).intersection(essay2coref_tagged.keys())
assert len(intersect) == len(essay2tagged.keys())
assert len(essay2tagged.keys()) > 1
assert len(essay2tagged.keys()) == len(essay2coref_tagged.keys())

## Examine Data Structure

### Notes on CoRef Datastructure
- Dictionary of esssays, keyed by name
- Each essay is a list of sentences
- Each sentence is a list of words
- words are mapped to a tag dict: Dict[str, Set[str]]]
  - tag dict - contains
    - NER tag (most are O - none)
    - POS tag
    - If a Co-Reference such as an anaphor (mostly pronouns)
      - COREF_PHRASE - phrase referred to by coref
      - COREF_REF - Id of referenced phrase
    - else if it is a phrase that is referenced:
      - COREF_ID - id of the co-reference, referenced in the COREF_REF tag

In [18]:
COREF_PHRASE = "COREF_PHRASE"
COREF_ID     = "COREF_ID"
COREF_REF    = "COREF_REF"

In [19]:
def find_coref_essay_sentence(essay2coref_tagged):
    for ename, list_sent in list(essay2coref_tagged.items()):
        for ix, sent in enumerate(list_sent):
            found_id = False
            found_ref = False
            for wd, tag_dict in sent:
                if COREF_ID in tag_dict:
                    found_id = True
                if COREF_REF in tag_dict:
                    found_ref = True
            if found_id and found_ref:
                return ename
    return None

ename = find_coref_essay_sentence(essay2coref_tagged)
ename

'EBA1415_AEKD_4_CB_ES-05568'

In [20]:
print(ename)
print()
wd_ix = -1
for sent in essay2coref_tagged[ename]: #[:matching_ix+1]:
    for wd, tag_dict in sent:
        wd_ix+=1
        copy = dict([(k,v) for k,v in tag_dict.items() if k in {COREF_ID, COREF_REF, COREF_PHRASE}])
        print(str(wd_ix).ljust(3), wd.ljust(20), copy if copy else "")
    print("*" * 80)

EBA1415_AEKD_4_CB_ES-05568

0   what                 
1   leads                
2   to                   
3   differences          
4   in                   
5   the                  
6   rates                
7   of                   
8   coral                
9   bleaching            
10  .                    
********************************************************************************
11  coral                
12  is                   
13  often                
14  mistaken             
15  for                  
16  a                    {'COREF_ID': {'1'}}
17  rock                 {'COREF_ID': {'1'}}
18  but                  
19  it                   {'COREF_REF': {'1'}, 'COREF_PHRASE': {'a_rock'}}
20  is                   
21  made                 
22  up                   
23  of                   
24  tiny                 
25  animals              
26  called               
27  polyps               
28  .                    
***************************************************

## Match CoRef Tagged to Consolidated Tagged Essays

In [21]:
def map_tagged_words_to_word_ixs(tagged_essay):

    tagged_wds = []
    taggedwd2sentixs = {}
    for sent_ix, sent in enumerate(tagged_essay.sentences):
        for wd_ix, (wd, tags) in enumerate(sent):
            taggedwd2sentixs[len(tagged_wds)] = (sent_ix, wd_ix)
            if wd == "\'\'":
                wd = "\""
            tagged_wds.append(wd)
    return tagged_wds, taggedwd2sentixs

def replace_underscore(mention):
    return set(map(lambda s: s.replace("_"," "), mention))

def map_mentions_to_word_ixs(coref_essay, keys):
    #TODO - fix this, it assume one mention per word, but we can have multiple
    coref_wd2_tags = []
    coref_ids_2_wd_ixs = defaultdict(list) # maps a coref id to a list of set of ixs
    for sent_ix, sent in enumerate(coref_essay):
        current_coref_ids = set()
        for wd_ix, (wd, tag_dict) in enumerate(sent):
            coref_wd2_tags.append((wd,tag_dict))

            wd_coref_ids = set()
            for k in keys:
                wd_coref_ids.update(tag_dict[k])

            for cref_id in wd_coref_ids:
                prev_ixs = coref_ids_2_wd_ixs[cref_id]
                # continuation of existing sequence
                wd_essay_ix = len(coref_wd2_tags)-1
                if cref_id in current_coref_ids:
                    if len(prev_ixs) == 0:
                        prev_ixs.append(set())
                    prev_ixs[-1].add(wd_essay_ix)
                else:
                    # else create a new set and add it
                    prev_ixs.append({wd_essay_ix})

            current_coref_ids = wd_coref_ids
    return coref_wd2_tags, coref_ids_2_wd_ixs

def map_words_between_essays(tagged_wds, coref_wd2_tags):
    errors = []

    ix_tagd, ix_coref = 0, 0
    ixtagd_2_ixcoref = {}
    ixcoref_2_ixtagd = {}

    while ix_tagd < (len(tagged_wds) - 1) and ix_coref < (len(coref_wd2_tags) - 1):
        wd_tagd = tagged_wds[ix_tagd]
        wd_coref, btag_dict = coref_wd2_tags[ix_coref]

        if wd_tagd == wd_coref or wd_tagd == "cannot" and wd_coref == "can":
            ixtagd_2_ixcoref[ix_tagd] = ix_coref
            ixcoref_2_ixtagd[ix_coref] = ix_tagd
            ix_tagd += 1
            ix_coref += 1
        else:
            # look ahead in wds2 for item that matches next a
            found_match = False
            for offseta, aa in enumerate(tagged_wds[ix_tagd: ix_tagd + 1 + SCAN_LENGTH]):
                for offsetb, (bb, _) in enumerate(coref_wd2_tags[ix_coref:ix_coref + 1 + SCAN_LENGTH]):
                    if aa == bb:
                        if offseta == offsetb:
                            for i in range(ix_tagd, ix_tagd + offseta):
                                if i not in ixtagd_2_ixcoref:
                                    ixtagd_2_ixcoref[i] = i

                        ix_tagd = ix_tagd + offseta
                        ix_coref = ix_coref + offsetb
                        ixtagd_2_ixcoref[ix_tagd] = ix_coref
                        ixcoref_2_ixtagd[ix_coref] = ix_tagd
                        found_match = True
                        break
                if found_match:
                    break
            if not found_match:
                errors.append((ename, wd_tagd, wd_coref, ix_tagd, ix_coref))
                break
    return ixtagd_2_ixcoref, ixcoref_2_ixtagd, errors

def map_wds_to_coref_ids(coref_ids_2_wd_ixs, ixcoref_2_ixtagd, coref_wd2_tags, tagged_wds, taggedwd2sentixs):
    sent_wdix_2_corefids = defaultdict(set)
    for coref_id, list_wd_ix_seq in coref_ids_2_wd_ixs.items():
        for wd_ixs in list_wd_ix_seq:

            first_ix = min(wd_ixs)
            is_fuzzy = False
            if first_ix not in ixcoref_2_ixtagd:
                while first_ix > 0 and first_ix not in ixcoref_2_ixtagd:
                    first_ix -= 1
                if first_ix not in ixcoref_2_ixtagd:
                    e_first_wd_ix = 0
                # one past last matching index
                else:
                    e_first_wd_ix = min(len(tagged_wds) - 1, ixcoref_2_ixtagd[first_ix] + 1)
                is_fuzzy = True
            else:
                e_first_wd_ix = ixcoref_2_ixtagd[first_ix]

            last_ix = max(wd_ixs)
            if last_ix not in ixcoref_2_ixtagd:
                while last_ix < len(coref_wd2_tags) and last_ix not in ixcoref_2_ixtagd:
                    last_ix += 1
                if last_ix not in ixcoref_2_ixtagd:
                    e_last_wd_ix = len(tagged_wds) - 1
                else:
                    e_last_wd_ix = max(0, ixcoref_2_ixtagd[last_ix] - 1)
                is_fuzzy = True
            else:
                e_last_wd_ix = ixcoref_2_ixtagd[last_ix]

            for e_wd_ix in range(e_first_wd_ix, e_last_wd_ix + 1):
                sent_ix, sent_wd_ix = taggedwd2sentixs[e_wd_ix]
                sent_wdix_2_corefids[(sent_ix, sent_wd_ix)].add(coref_id)
    return sent_wdix_2_corefids


In [10]:
updated_essays = []

In [11]:
coref_essay  = essay2coref_tagged[ename]
tagged_essay = essay2tagged[ename]

tagged_wds, taggedwd2sentixs = map_tagged_words_to_word_ixs(tagged_essay)
coref_wd2_tags, coref_ids_2_wd_ixs = map_mentions_to_word_ixs(coref_essay, keys={COREF_ID, COREF_REF})

In [12]:
ixtagd_2_ixcoref, ixcoref_2_ixtagd, errors = map_words_between_essays(tagged_wds, coref_wd2_tags)
if errors:
    # Print errors
    for ename, wd_tagd, wd_coref, ix_tagd, ix_coref in errors:
        failed_cnt += 1
        print("Failed: " + ename, wd_tagd, wd_coref, ix_tagd, ix_coref)

In [22]:
sent_wdix_2_corefids = map_wds_to_coref_ids(coref_ids_2_wd_ixs=coref_ids_2_wd_ixs,
                                                ixcoref_2_ixtagd=ixcoref_2_ixtagd,
                                                coref_wd2_tags=coref_wd2_tags,
                                                tagged_wds=tagged_wds,
                                                taggedwd2sentixs=taggedwd2sentixs)

In [23]:
predicted_corefids_sentences = []
for sent_ix, sent in enumerate(tagged_essay.sentences):
    changed_ix = -1

    predicted_coref_ids_wds = []
    predicted_corefids_sentences.append(predicted_coref_ids_wds)

    for wd_ix in range(len((sent))):
        wd_coref_ids = sent_wdix_2_corefids[(sent_ix, wd_ix)]
        predicted_coref_ids_wds.append(wd_coref_ids)

tagged_essay.predicted_corefids = predicted_corefids_sentences

In [ ]:
updated_essays.append(tagged_essay)

In [14]:
from pprint import pprint
pprint(dict(coref_ids_2_wd_ixs.items()))

{'1': [{6, 7}, {10}],
 '2': [{68, 69, 70, 71, 72, 73, 74}, {83, 84}, {101}],
 '3': [{42, 43, 44}, {46, 47}]}


In [16]:
coref_essay

[[('well',
   defaultdict(set,
               {'COREF_ID': set(),
                'COREF_REF': set(),
                'NER': {'O'},
                'POS': {'UH'}})),
  ('based',
   defaultdict(set,
               {'COREF_ID': set(),
                'COREF_REF': set(),
                'NER': {'O'},
                'POS': {'VBN'}})),
  ('on',
   defaultdict(set,
               {'COREF_ID': set(),
                'COREF_REF': set(),
                'NER': {'O'},
                'POS': {'IN'}})),
  ('what',
   defaultdict(set,
               {'COREF_ID': set(),
                'COREF_REF': set(),
                'NER': {'O'},
                'POS': {'WP'}})),
  ('i',
   defaultdict(set,
               {'COREF_ID': set(),
                'COREF_REF': set(),
                'NER': {'O'},
                'POS': {'FW'}})),
  ('read',
   defaultdict(set,
               {'COREF_ID': set(),
                'COREF_REF': set(),
                'NER': {'O'},
                'POS': {'VB'}})),
  ('the

In [17]:
for sent_ix, sent in enumerate(tagged_essay.sentences):
    
    pred_tags = tagged_essay.pred_tagged_sentences[sent_ix]
    corefid_sent = tagged_essay.predicted_corefids[sent_ix]
    for wd_ix in range(len(sent)):
        wd = sent[wd_ix][0]
        
        cref_ids = corefid_sent[wd_ix]
        tag = pred_tags[wd_ix]
        print(wd.ljust(20), tag.ljust(10), cref_ids)

well                 Empty      set()
based                Empty      set()
on                   Empty      set()
what                 Empty      set()
i                    Empty      set()
read                 Empty      set()
the                  Empty      {'1'}
corals               Empty      {'1'}
are                  Empty      set()
INFREQUENT           Empty      set()
their                Empty      {'1'}
colors               50         set()
,                    Empty      set()
coral                50         set()
bleaching            50         set()
are                  Empty      set()
a                    Empty      set()
serious              Empty      set()
problem              Empty      set()
with                 Empty      set()
a                    Empty      set()
serious              Empty      set()
impact               Empty      set()
on                   Empty      set()
the                  Empty      set()
worlds               Empty      set()
coral       